In [ ]:
import typing
import torch
import dill
from torch import nn
from dataclasses import dataclass
from avalanche.training.plugins import StrategyPlugin


%load_ext autoreload
%autoreload 2
from experiment import Experiment, BaseHyperParameters

In [60]:
from network.module.dropout import ConditionedDropout
from network.nn_traits import HasConditionedDropout

class Network(nn.Module, HasConditionedDropout):
    def __init__(self, input_size: int, p_drop: float) -> None:
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(input_size, 10),
            # nn.Dropout(p_drop),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.net(x)

In [61]:
from avalanche.training.strategies.cumulative import Cumulative
from avalanche.training.plugins.synaptic_intelligence import SynapticIntelligencePlugin
from avalanche.benchmarks.generators import nc_benchmark
from torch.nn.functional import normalize

@dataclass
class HyperParams(BaseHyperParameters):
    label: str
    dataset: str
    n_experiences: int
    latent_dim: int
    p_drop: float

class MyExperiment(Experiment):

    hp: HyperParams
    network: Network

    def __init__(self, hp: HyperParams) -> None:
        super().__init__(hp)

    def make_network(self) -> nn.Module:
        return Network(self.hp.latent_dim, self.hp.p_drop)

    # def add_plugins(self) -> typing.Sequence[StrategyPlugin]:
    #     return [SynapticIntelligencePlugin(500)]

    def make_optimizer(self, parameters) -> torch.optim.Optimizer:
        optimizer = torch.optim.SGD(parameters, self.hp.lr, weight_decay=0.01)
        return optimizer

    def before_training_exp(self, strategy: 'BaseStrategy', **kwargs):
        experience = self.strategy.clock.train_exp_counter
        print("activating group", experience)
        self.network.set_active_group(experience)


    def make_scenario(self):
        with open(self.hp.dataset, "rb") as f:
            dataset = dill.load(f)

        dataset.trainset.latent_codes = normalize(dataset.trainset.latent_codes)
        dataset.testset.latent_codes = normalize(dataset.testset.latent_codes)

        scenario = nc_benchmark(
            dataset.trainset, dataset.testset, n_experiences=self.hp.n_experiences, shuffle=True, seed=1234,
            task_labels=False
        )
        return scenario

In [ ]:
import numpy as np

def make_random_hp_search(dataset:str, label:str, latent_dim:int, n_experiences:int, p_drop:float) -> typing.Callable[[], HyperParams]:
    def _random():
        return HyperParams(
            lr=np.random.uniform(0.1, 0.0001),
            train_mb_size=64,
            train_epochs=10,
            eval_mb_size=1024,
            eval_every=-1,
            label=label,
            dataset=dataset,
            n_experiences=n_experiences,
            latent_dim=latent_dim,
            p_drop=0.5,
            device="cuda"
        )

    return _random

def train_n_times(hp_func, x):
    for _ in range(x):
        MyExperiment(hp_func()).train()

n = 1
train_n_times(make_random_hp_search("notebooks/L128_FMNIST_SSAE.pkl", "Continual", 128, 5, 0.0), n)
# train_n_times(make_random_hp_search("notebooks/L016_FMNIST.pkl", "Dropout", 16, 5, 0.5), n)
# train_n_times(make_random_hp_search("notebooks/L032_FMNIST.pkl", "Dropout", 32, 5, 0.5), n)
# train_n_times(make_random_hp_search("notebooks/L064_FMNIST.pkl", "Dropout", 64, 5, 0.5), n)
# train_n_times(make_random_hp_search("notebooks/L128_FMNIST.pkl", "Dropout", 128, 5, 0.5), n)
# train_n_times(make_random_hp_search("notebooks/L256_FMNIST.pkl", "Dropout", 256, 5, 0.5), n)

In [62]:
MyExperiment(
    HyperParams(
            lr=0.01,
            train_mb_size=64,
            train_epochs=2,
            eval_mb_size=1024,
            eval_every=-1,
            label="",
            dataset="notebooks/L128_FMNIST_SSAE.pkl",
            n_experiences=5,
            latent_dim=128,
            p_drop=0.1,
            device="cuda"
    )
).train()

Start of experience:  0
Current Classes:      [4, 5]
activating group 0
Start of experience:  1
Current Classes:      [8, 2]
activating group 1
Start of experience:  2
Current Classes:      [9, 6]
activating group 2
Start of experience:  3
Current Classes:      [1, 7]
activating group 3
Start of experience:  4
Current Classes:      [0, 3]
activating group 4


[{'Loss_MB/train_phase/train_stream/Task000': 1.1382803916931152,
  'RunningLoss_Epoch/train_phase/train_stream/Task000': 1.345141284942627,
  'training_loss/train_phase/train_stream/Task000': 1.627522844950358,
  'Loss_Epoch/train_phase/train_stream/Task000': 1.345141284942627,
  'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.9925,
  'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 1.147309923171997,
  'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.994,
  'Loss_Stream/eval_phase/test_stream/Task000': 1.147309923171997,
  'Top1_Acc_Stream/eval_phase/test_stream/Task000': 0.994,
  'Accuracy_On_Trained_Experiences/eval_phase/test_stream/Task000': 0.994,
  'ConfusionMatrix_Stream/eval_phase/test_stream': <avalanche.evaluation.metric_results.AlternativeValues at 0x7f3438a6b0d0>,
  'StreamForgetting/eval_phase/test_stream': 0.0},
 {'Loss_MB/train_phase/train_stream/Task000': 1.3308472633361816,
  'RunningLoss_Epoch/train_phase/train_stream/Task000': 1.6167220439910888,
  'tr